# MED8 ADSSE - Design Patterns

For a more comprehensive resource on design patterns in general, and specific examples in multiple languages including python, please check out [this resource](https://refactoring.guru/design-patterns/python)

## Singleton
Despite being one of the original Group of Four design patterns, the Singleton is declining in popularity. Many developers consider it an "anti-pattern", as it is often used in a way that flys in the face of SOLID principles. However the issue lies not necessarily with the singleton itself, but rather with the way in which it is used.

Below is a simple implementation of a Singleton. This is a class which contains a variable ```__instance``` (note the meaning of the leading ```__``` is somewhat complicated, read about underscores in Python [here](https://dbader.org/blog/meaning-of-underscores-in-python)). This is functionally similar to a private variable.

The ```getInstance()``` method checks if ```__instance``` has already been assigned. If not, it will call the constructor. This is the core principal of the Singleton, as it is responsible for calling its own constructor, and limits itself to one instance.

Note that this is a simple implementation, and is not threadsafe. In a multithread environment it would be possible to create multiple instances of this singleton, so more checks and use of locks would be needed.

In [32]:
class Singleton():
    __instance = None
    
    @staticmethod
    def getInstance():
        if Singleton.__instance == None:
            Singleton()
        return Singleton.__instance
    
    def __init__(self):
        if Singleton.__instance != None:
            raise Exception("Singleton cannot have more than one instance")
        else:
            Singleton.__instance = self


In [33]:
    s1 = Singleton.getInstance()
    s2 = Singleton.getInstance()

    if id(s1) == id(s2):
        print("Singleton works, both variables contain the same instance.")
    else:
        print("Singleton failed, variables contain different instances.")

Singleton works, both variables contain the same instance.


As we can see, despite making two objects of the Singleton class, s1 and s2, they are the same instance of the class.

We can make this a bit more explicit by creating a new type of singleton with some functionality. The new class ```CounterSingleton``` uses the same logic as before, but also includes an integer value, an accesor method, and a method which simply increments the value.

In [34]:
class CounterSingleton():
    
    __instance = None
    __value = 0
    
    @staticmethod
    def getInstance():
        if CounterSingleton.__instance == None:
            CounterSingleton()
        return CounterSingleton.__instance
    
    def __init__(self):
        if CounterSingleton.__instance != None:
            raise Exception("Singleton cannot have more than one instance")
        else:
            CounterSingleton.__instance = self  
            __value = 0

    # Expanding on the previous Singleton with a s
    @property
    def value(self):
        return self.__value
    
    def count(self):
        self.__value = self.__value + 1

Once more, we will test the basic funcitonality and ensure that this singleton is successfully limited to one instance, by creating two objects.

In [35]:
c1 = CounterSingleton.getInstance()
c2 = CounterSingleton.getInstance()

if id(c1) == id(c2):
    print("Singleton works, both variables contain the same instance.")
else:
    print("Singleton failed, variables contain different instances.")

Singleton works, both variables contain the same instance.


Let's try see the effect of having just one instance, by using one of these objects to print the value, and the other one to count.

In [36]:
print("c1 starting value: " + str(c1.value))
c2.count()
print("c1 value after c2.count(): " + str(c1.value))

c1 starting value: 0
c1 value after c2.count(): 1


Going further, since ```getInstance``` is a static method, we don't need to create an object to use the functionality. Here we access the value directly from the singleton class and store it in a varaible ```x```. Note that this value no longer changes when using the ```count()``` method, since it is stored in ```x``` and not in ```__value```.

In [37]:
x = CounterSingleton.getInstance().value
CounterSingleton.getInstance().count()
print(x)
print(CounterSingleton.getInstance().value)

1
2


## a note on the use of the _dunder_ for encapsulation
As mentioned above, the _dunder_ (double underscore, ```__```) prefix acts functionally as a private variable, as attributes with this prefix cannot be accessed directly. The code in the following block throws an attribute error, because it cannot find the ```__instance``` attribute.

In [7]:
print(s1.__instance)


AttributeError: 'Singleton' object has no attribute '__instance'

However, don't assume that this is the same as private encapsulation. Notice that the error says ```'Singleton' object has no attribute '__instance'```, rather than saying that it is inaccesible. This is because it is not private, but rather the dunder has flagged it to be renamed based on the class. This is known as _name mangling_. The attribute exists, but instead of being called ```.__attribute``` it is called  ```._classname__attribute```:

In [8]:
print(s1._Singleton__instance)

## Observer



In other languages such as Java or C#, this pattern is mostly implemented using Interfaces. In this python implementation, the interfaces are replaces with the abstract base classes (ABCs) ```Subject``` and ```Observer```.

In [22]:
from abc import ABC, abstractmethod
from random import randrange
from typing import List


class Subject(ABC):
    """
    The Subject interface declares a set of methods for managing subscribers.
    """

    @abstractmethod
    def attach(self, observer: Observer):
        """
        Attach an observer to the subject.
        """
        pass

    @abstractmethod
    def detach(self, observer: Observer):
        """
        Detach an observer from the subject.
        """
        pass

    @abstractmethod
    def notify(self):
        """
        Notify all observers about an event.
        """
        pass

class Observer(ABC):
    """
    The Observer interface declares the update method, used by subjects.
    """

    @abstractmethod
    def update(self, subject: Subject):
        """
        Receive update from subject.
        """
        pass


A ```ConcreteSubject``` is using the ABC. Since python supports multi-inheritance, this is equivalent to implementing an interface, and the subject may inheret from other base classes too.

For the purpose of this example, the ```ConcreteSubject``` has a ```_state``` attribute, which represents the data that the observers are interested in. This could be player health in a game for example, or content for a mailing list. We are using just a simple integer value, but it could be a much more complex set of data.

The key functionality of the Observer pattern here is the implemetnation of the functions from the interface, ```attach()```, ```detach()```, and ```notify```, as well as the list of observers.

The ```some_business_logic``` function is an example of the subject's behaviour, and is simply used to change the ```_state_``` and to determine when ```notify``` is called.

In [23]:
class ConcreteSubject(Subject):
    """
    The Subject owns some important state and notifies observers when the state
    changes.
    """

    _state: int = None
    """
    For the sake of simplicity, the Subject's state, essential to all
    subscribers, is stored in this variable.
    """

    _observers: List[Observer] = []
    """
    List of subscribers. In real life, the list of subscribers can be stored
    more comprehensively (categorized by event type, etc.).
    """

    def attach(self, observer: Observer):
        print("Subject: Attached an observer.")
        self._observers.append(observer)

    def detach(self, observer: Observer):
        self._observers.remove(observer)

    """
    The subscription management methods.
    """

    def notify(self) -> None:
        """
        Trigger an update in each subscriber.
        """

        print("Subject: Notifying observers...")
        for observer in self._observers:
            observer.update(self)

    def some_business_logic(self):
        """
        Usually, the subscription logic is only a fraction of what a Subject can
        really do. Subjects commonly hold some important business logic, that
        triggers a notification method whenever something important is about to
        happen (or after it).
        """

        print("\nSubject: I'm doing something important.")
        self._state = randrange(0, 10)

        print(f"Subject: My state has just changed to: {self._state}")
        self.notify()


Similarly, a number of classes are created using the Observer interface. They the exact contents of the ```update``` method will vary from each observer, but the key element here is that when the subject calls ```notify```, all observers in the list will have their ```update``` methods called. 

In [24]:
class ConcreteObserverA(Observer):
    def update(self, subject: Subject):
        if subject._state < 3:
            print("ConcreteObserverA: Reacted to the event")


class ConcreteObserverB(Observer):
    def update(self, subject: Subject):
        if subject._state == 0 or subject._state >= 2:
            print("ConcreteObserverB: Reacted to the event")


if __name__ == "__main__":
    # The client code.

    subject = ConcreteSubject()

    observer_a = ConcreteObserverA()
    subject.attach(observer_a)

    observer_b = ConcreteObserverB()
    subject.attach(observer_b)

    subject.some_business_logic()
    subject.some_business_logic()

    subject.detach(observer_a)

    subject.some_business_logic()

Subject: Attached an observer.
Subject: Attached an observer.

Subject: I'm doing something important.
Subject: My state has just changed to: 5
Subject: Notifying observers...
ConcreteObserverB: Reacted to the event

Subject: I'm doing something important.
Subject: My state has just changed to: 3
Subject: Notifying observers...
ConcreteObserverB: Reacted to the event

Subject: I'm doing something important.
Subject: My state has just changed to: 5
Subject: Notifying observers...
ConcreteObserverB: Reacted to the event


Let's try the same thing with a game example. Here we will make ```PlayerHealth``` into the subject, and other systems such as the ```AudioManager```, ```VisualEffects```, and ```EnemyAI``` will all act as observers. The subject will send a notification to all observers when it takes damage. Each of them will have their own behaviour based on what they are interested in doing with the notification.

In [31]:
class PlayerHealth(Subject):
    _health: int
    _is_alive: bool
    
    def __init__(self):
        self._health = 100
        self._is_alive = True

    _observers: List[Observer] = []

    def attach(self, observer: Observer):
        self._observers.append(observer)

    def detach(self, observer: Observer):
        self._observers.remove(observer)

    def notify(self):
        for observer in self._observers:
            observer.update(self)

    def take_damage(self, damage):


        print(">>>Player took " + str(damage) + " damage")
        self._health = self._health - damage
        if(self._health < 0):
            self._is_alive = False

        self.notify()

class AudioManager(Observer):
    def update(self, subject: Subject):
        if not subject._is_alive:
            print("Audio: Play death sfx")
        elif subject._health < 50:
            print("Audio: Play low health sfx")
        else:
            print("Audio: Play regular ambiance")
                
class VisualEffects(Observer):
    def update(self, subject: Subject):
        if subject._is_alive:
            print("VFX: Set blood splatter strength to " + str(100-subject._health))
        else:
            print("VFX: Show death screen")

class EnemyAI(Observer):
    def update(self, subject:Subject):
        if not subject._is_alive:
            print("Enemies: Celebrate victory")
        elif subject._health < 40:
            print("Enemies: Stop attacking")
        else:
            print("Enemies: Become aggressive")
 

if __name__ == "__main__":
    # The client code.

    playerHealth = PlayerHealth()

    sfx = AudioManager()
    playerHealth.attach(sfx)

    vfx = VisualEffects()
    playerHealth.attach(vfx)

    enemies = EnemyAI()
    playerHealth.attach(enemies)
    
    playerHealth.take_damage(10)

    playerHealth.take_damage(20)
    playerHealth.take_damage(40)
    playerHealth.take_damage(40)


>>>Player took 10 damage
Audio: Play regular ambiance
VFX: Set blood splatter strength to 10
Enemies: Become aggressive
>>>Player took 20 damage
Audio: Play regular ambiance
VFX: Set blood splatter strength to 30
Enemies: Become aggressive
>>>Player took 40 damage
Audio: Play low health sfx
VFX: Set blood splatter strength to 70
Enemies: Stop attacking
>>>Player took 40 damage
Audio: Play death sfx
VFX: Show death screen
Enemies: Celebrate victory
